In [7]:
import numpy as np

import pandas as pd
import sklearn
from sklearn import preprocessing

import bokeh
#from bokeh.charts import BoxPlot
import bokeh.plotting
from bokeh.plotting import ColumnDataSource
from bokeh.models import LabelSet
from bokeh.models import FuncTickFormatter
from bokeh.io import export_svgs

import bokeh.io
bokeh.io.output_notebook()

Loading BokehJS ...

In [8]:
#load each run into a separate dataframe
df1 = pd.read_excel('190926_Deuterium_Transfer_Peak_Areas.xlsx', comment='#')
df2 = pd.read_excel('191007_Deuterium_Transfer_Peak_Areas.xlsx', comment='#')
df3 = pd.read_excel('191010_Deuterium_Transfer_Peak_Areas.xlsx', comment='#')
df4 = pd.read_excel('191015_Deuterium_Transfer_Peak_Areas.xlsx', comment='#')
df5 = pd.read_excel('191020_Deuterium_Transfer_Peak_Areas.xlsx', comment='#')
df6 = pd.read_excel('191021_Deuterium_Transfer_Peak_Areas.xlsx', comment='#')
df7 = pd.read_excel('191022_Deuterium_Transfer_Peak_Areas.xlsx', comment='#')
df8 = pd.read_excel('191025_Deuterium_Transfer_Peak_Areas.xlsx', comment='#')
df9 = pd.read_excel('191027_Deuterium_Transfer_Peak_Areas.xlsx', comment='#')
df10 = pd.read_excel('191031_Deuterium_Transfer_Peak_Areas.xlsx', comment='#')
df11 = pd.read_excel('191106_Deuterium_Transfer_Peak_Areas.xlsx', comment='#')

In [9]:
#add a column to each dataframe listing the date
df1['date'] = '190926'
df2['date'] = '191007'
df3['date'] = '191010'
df4['date'] = '191015'
df5['date'] = '191020'
df6['date'] = '191021'
df7['date'] = '191022'
df8['date'] = '191025'
df9['date'] = '191027'
df10['date'] = '191031'
df11['date'] = '191106'

#merge data frames into one dataframe
df=df1.append(df2).append(df3).append(df4).append(df5).append(df6).append(df7).append(df8).append(df9).append(df10).append(df11)
df.tail()

,Species,WellNumber,Peak ID,Ret Time,Start Tm,End Tm,m/z,Area,Area Percent,Height,Height Percent,A/H,Type,Outlier,date
43,L,8,D50C24,9.553,9.540,9.570,TIC,694,0.02,898,0.02,0.77,U,N,191106
44,L,8,D62C30,12.355,12.310,12.355,TIC,6249,0.16,5624,0.12,1.11,U,N,191106
45,D,8,C18,7.426,7.400,7.445,TIC,3770792,99.52,4782847,99.67,0.79,U,N,191106
46,D,8,D50C24,9.520,9.495,9.520,TIC,703,0.02,0,0.00,0.00,U,N,191106
47,D,8,D62C30,12.355,12.310,12.355,TIC,17543,0.46,15860,0.33,1.11,U,N,191106


In [10]:
#use the area of the c18 peak to calculate the amount of DCHC present on each insect

#grab the values of each of the c18 rows
c18_Areas = np.repeat(df['Area'].loc[df['Peak ID'] == 'C18'],3).reset_index().drop("index", axis=1)

#divide the measured area of each row by its corresponding c18 value to get the ratio 
#of the given compounds measured intensity to the measured intensity of the C18 standard
df['Normalized Area'] = np.divide(df['Area'],c18_Areas['Area'])
#multiply by 25ng, the amount of C18 in the 1 µL injected
df['Hydrocarbon amount'] = df['Normalized Area']*25

In [11]:
#Remove any outliers, specified by 'Y' in the Outlier column
df = df.loc[df['Outlier'] == 'N']

In [12]:
df['Identifier']=''
for j,date in enumerate(np.unique(df['date'])):
    df_date = df.loc[df['date']==date]
    for i in range(8):
        curSample = df_date.loc[df_date['WellNumber'] == i+1]
        if (any((curSample['Species'] == 'S') & (curSample['Type'] == 'U'))):
            df['Identifier'].loc[(df['date']==date) & (df['WellNumber']==i+1) & (df['Species'] == 'L')] = 'LSU'
            df['Identifier'].loc[(df['date']==date) & (df['WellNumber']==i+1) & (df['Species'] == 'S')] = 'SU'
                    
        if (any((curSample['Species'] == 'S') & (curSample['Type'] == 'L'))):
            df['Identifier'].loc[(df['date']==date) & (df['WellNumber']==i+1) & (df['Species'] == 'L')] = 'LSL'
            df['Identifier'].loc[(df['date']==date) & (df['WellNumber']==i+1) & (df['Species'] == 'S')] = 'SL'
        
        
        if (any((curSample['Species'] == 'P') & (curSample['Type'] == 'U'))):
            df['Identifier'].loc[(df['date']==date) & (df['WellNumber']==i+1) & (df['Species'] == 'L')] = 'LPU'
            df['Identifier'].loc[(df['date']==date) & (df['WellNumber']==i+1) & (df['Species'] == 'P')] = 'PU'
        
        if (any((curSample['Species'] == 'P') & (curSample['Type'] == 'L'))):
            df['Identifier'].loc[(df['date']==date) & (df['WellNumber']==i+1) & (df['Species'] == 'L')] = 'LPL'
            df['Identifier'].loc[(df['date']==date) & (df['WellNumber']==i+1) & (df['Species'] == 'P')] = 'PL'
            
            
        if (any((curSample['Species'] == 'D') & (curSample['Type'] == 'U'))):
            df['Identifier'].loc[(df['date']==date) & (df['WellNumber']==i+1) & (df['Species'] == 'L')] = 'LDU'
            df['Identifier'].loc[(df['date']==date) & (df['WellNumber']==i+1) & (df['Species'] == 'D')] = 'DU'
        
        if (any((curSample['Species'] == 'D') & (curSample['Type'] == 'L'))):
            df['Identifier'].loc[(df['date']==date) & (df['WellNumber']==i+1) & (df['Species'] == 'L')] = 'LDL'
            df['Identifier'].loc[(df['date']==date) & (df['WellNumber']==i+1) & (df['Species'] == 'D')] = 'DL'
        

/Users/TomNaragon/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [13]:
df.head()

,Species,WellNumber,Peak ID,Ret Time,Start Tm,End Tm,m/z,Area,Area Percent,Height,Height Percent,A/H,Type,Outlier,date,Normalized Area,Hydrocarbon amount,Identifier
0,L,1,C18,7.435,7.405,7.470,TIC,5589973,93.17,5956210,95.09,0.94,L,N,190926,1.000000,25.000000,LSL
1,L,1,D50C24,9.498,9.475,9.525,TIC,231950,3.87,222378,3.55,1.04,L,N,190926,0.041494,1.037348,LSL
2,L,1,D62C30,12.350,12.330,12.375,TIC,177959,2.97,85013,1.36,2.09,L,N,190926,0.031835,0.795885,LSL
3,S,1,C18,7.434,7.410,7.455,TIC,6037087,99.11,6123349,99.37,0.99,L,N,190926,1.000000,25.000000,SL
4,S,1,D50C24,9.497,9.475,9.525,TIC,37683,0.62,31671,0.51,1.19,L,N,190926,0.006242,0.156048,SL


In [14]:
df_C24 = df.loc[df['Peak ID'] == 'D50C24']
df_C30 = df.loc[df['Peak ID'] == 'D62C30']

In [22]:
Identifiers = ['LSU','SU','LSL','SL','LPU','PU','LPL','PL','LDU','DU','LDL','DL',]
Labels={'LSU': '','SU': 'Sceptobius Control','LSL': '','SL': 'Sceptobius Treated',
        'LPU': '','PU': 'Platyusa Control','LPL': '','PL': 'Platyusa Treated',
        'LDU': '','DU': 'Dalotia Control','LDL': '','DL': 'Dalotia Treated'}

np.random.seed(666)  

p = bokeh.plotting.figure(plot_width=800,
                          plot_height=600,
                          title='C24',
                          x_range=Identifiers,
                          y_axis_label='ng D50C24',
                          y_axis_type='log')

colors=['#494949','#5F56FF','#494949','#5F56FF','#494949','#C42F2F','#494949','#C42F2F','#494949','#832161','#494949','#832161']


groups = df_C24.groupby('Identifier')
q1 = groups.quantile(q=0.25)
q2 = groups.quantile(q=0.5)
q3 = groups.quantile(q=0.75)
iqr = q3 - q1
upper = q3 + 1.5*iqr
lower = q1 - 1.5*iqr

# find the outliers for each category
def outliers(group):
    cat = group.name
    return group[(group['Hydrocarbon amount'] > upper.loc[cat]['Hydrocarbon amount']) | (group['Hydrocarbon amount'] < lower.loc[cat]['Hydrocarbon amount'])]['Hydrocarbon amount']
out = groups.apply(outliers).dropna()

# prepare outlier data for plotting, we need coordinates for every outlier.
if not out.empty:
    outx = []
    outy = []
    for keys in out.index:
        outx.append(keys[0])
        outy.append(out.loc[keys[0]].loc[keys[1]])

# if no outliers, shrink lengths of stems to be no longer than the minimums or maximums
qmin = groups.quantile(q=0.00)
qmax = groups.quantile(q=1.00)
upper['Hydrocarbon amount'] = [min([x,y]) for (x,y) in zip(list(qmax.loc[:,'Hydrocarbon amount']),upper['Hydrocarbon amount'])]
lower['Hydrocarbon amount'] = [max([x,y]) for (x,y) in zip(list(qmin.loc[:,'Hydrocarbon amount']),lower['Hydrocarbon amount'])]

# stems
p.segment(q3.reset_index()['Identifier'], upper['Hydrocarbon amount'], q3.reset_index()['Identifier'], q3['Hydrocarbon amount'], line_color="black")
p.segment(q3.reset_index()['Identifier'], lower['Hydrocarbon amount'], q3.reset_index()['Identifier'], q1['Hydrocarbon amount'], line_color="black")

#boxes
p.vbar(q3.reset_index()['Identifier'], 0.7, q2['Hydrocarbon amount'], q3['Hydrocarbon amount'], fill_color=None, line_color="black")
p.vbar(q3.reset_index()['Identifier'], 0.7, q1['Hydrocarbon amount'], q2['Hydrocarbon amount'], fill_color=None, line_color="black")

# whiskers (almost-0 height rects simpler than segments)
p.vbar(q3.reset_index()['Identifier'], top=lower['Hydrocarbon amount'],bottom=lower['Hydrocarbon amount'], width=0.2, line_color="black")
p.vbar(q3.reset_index()['Identifier'], top=upper['Hydrocarbon amount'],bottom=upper['Hydrocarbon amount'], width=0.2, line_color="black")

for j,date in enumerate(np.unique(df_C24['date'])):
    for _,i in enumerate(np.unique(df_C24['WellNumber'].loc[df_C24['date']==date])):
        
        data = {'identifier': df_C24['Identifier'].loc[(df_C24['date']==date)&(df_C24['WellNumber']==i)].values,
                'hydrocarbon amount': df_C24['Hydrocarbon amount'].loc[(df_C24['date']==date)&(df_C24['WellNumber']==i)].values}

        source = ColumnDataSource(data=data)

        offsetVal=(np.random.rand(1)[0]-0.5)*0.5
        p.line(bokeh.transform.dodge('identifier',  offsetVal,  range=p.x_range),
               'hydrocarbon amount',
               source=source,
               color='black',
               alpha=0.3)
        
        p.circle(bokeh.transform.dodge('identifier',  offsetVal,  range=p.x_range),
               'hydrocarbon amount',
               source=source,
               color=bokeh.transform.factor_cmap('identifier',colors,Identifiers),
               alpha=0.6,
               size=7)
        

p.xgrid.visible = False
p.ygrid.visible = False

# Add custom axis

p.xaxis.formatter = FuncTickFormatter(code="""
var labels = %s;
return labels[tick];
""" %Labels)

p.output_backend = "svg"
export_svgs(p, filename="DCHC_transfer_C24.svg")

bokeh.io.show(p)

In [16]:
Identifiers = ['LSU','SU','LSL','SL','LPU','PU','LPL','PL','LDU','DU','LDL','DL',]
Labels={'LSU': '','SU': 'Sceptobius Control','LSL': '','SL': 'Sceptobius Treated',
        'LPU': '','PU': 'Platyusa Control','LPL': '','PL': 'Platyusa Treated',
        'LDU': '','DU': 'Dalotia Control','LDL': '','DL': 'Dalotia Treated'}

np.random.seed(666)  

p = bokeh.plotting.figure(plot_width=800,
                          plot_height=600,
                          title='C30',
                          x_range=Identifiers,
                          y_axis_label='ng D62C30',
                          y_axis_type='log')

colors=['#494949','#5F56FF','#494949','#5F56FF','#494949','#C42F2F','#494949','#C42F2F','#494949','#832161','#494949','#832161']

groups = df_C30.groupby('Identifier')
q1 = groups.quantile(q=0.25)
q2 = groups.quantile(q=0.5)
q3 = groups.quantile(q=0.75)
iqr = q3 - q1
upper = q3 + 1.5*iqr
lower = q1 - 1.5*iqr

# find the outliers for each category
def outliers(group):
    cat = group.name
    return group[(group['Hydrocarbon amount'] > upper.loc[cat]['Hydrocarbon amount']) | (group['Hydrocarbon amount'] < lower.loc[cat]['Hydrocarbon amount'])]['Hydrocarbon amount']
out = groups.apply(outliers).dropna()

# prepare outlier data for plotting, we need coordinates for every outlier.
if not out.empty:
    outx = []
    outy = []
    for keys in out.index:
        outx.append(keys[0])
        outy.append(out.loc[keys[0]].loc[keys[1]])

# if no outliers, shrink lengths of stems to be no longer than the minimums or maximums
qmin = groups.quantile(q=0.00)
qmax = groups.quantile(q=1.00)
upper['Hydrocarbon amount'] = [min([x,y]) for (x,y) in zip(list(qmax.loc[:,'Hydrocarbon amount']),upper['Hydrocarbon amount'])]
lower['Hydrocarbon amount'] = [max([x,y]) for (x,y) in zip(list(qmin.loc[:,'Hydrocarbon amount']),lower['Hydrocarbon amount'])]

# stems
p.segment(q3.reset_index()['Identifier'], upper['Hydrocarbon amount'], q3.reset_index()['Identifier'], q3['Hydrocarbon amount'], line_color="black")
p.segment(q3.reset_index()['Identifier'], lower['Hydrocarbon amount'], q3.reset_index()['Identifier'], q1['Hydrocarbon amount'], line_color="black")

#boxes
p.vbar(q3.reset_index()['Identifier'], 0.7, q2['Hydrocarbon amount'], q3['Hydrocarbon amount'], fill_color=None, line_color="black")
p.vbar(q3.reset_index()['Identifier'], 0.7, q1['Hydrocarbon amount'], q2['Hydrocarbon amount'], fill_color=None, line_color="black")

# whiskers (almost-0 height rects simpler than segments)
p.vbar(q3.reset_index()['Identifier'], top=lower['Hydrocarbon amount'],bottom=lower['Hydrocarbon amount'], width=0.2, line_color="black")
p.vbar(q3.reset_index()['Identifier'], top=upper['Hydrocarbon amount'],bottom=upper['Hydrocarbon amount'], width=0.2, line_color="black")

for j,date in enumerate(np.unique(df_C30['date'])):
    for _,i in enumerate(np.unique(df_C30['WellNumber'].loc[df_C30['date']==date])):
        
        data = {'identifier': df_C30['Identifier'].loc[(df_C30['date']==date)&(df_C30['WellNumber']==i)].values,
                'hydrocarbon amount': df_C30['Hydrocarbon amount'].loc[(df_C30['date']==date)&(df_C30['WellNumber']==i)].values}

        source = ColumnDataSource(data=data)

        offsetVal=(np.random.rand(1)[0]-0.5)*0.5
#         p.line(bokeh.transform.dodge('identifier',  offsetVal,  range=p.x_range),
#                'hydrocarbon amount',
#                source=source,
#                color='black',
#                alpha=0.3)
        
        p.circle(bokeh.transform.dodge('identifier',  offsetVal,  range=p.x_range),
               'hydrocarbon amount',
               source=source,
               color=bokeh.transform.factor_cmap('identifier',colors,Identifiers),
               alpha=0.6,
               size=7)
        

p.xgrid.visible = False

# Add custom axis

p.xaxis.formatter = FuncTickFormatter(code="""
var labels = %s;
return labels[tick];
""" %Labels)

bokeh.io.show(p)

In [17]:
p = bokeh.plotting.figure(plot_width=600,
                          plot_height=600,
                          title='C24',
                          x_axis_label='Species',
                          y_axis_label='ng D50C24',
                          y_axis_type='log')
colors=['red','orange','green','blue','purple','black','grey','pink','indigo']
for j,date in enumerate(np.unique(df_C24['date'])):
    df_C24_date = df_C24.loc[df_C24['date']==date]
    for i in range(8):
        curSample = df_C24_date.loc[df_C24_date['WellNumber'] == i+1]
        if (any((curSample['Species'] == 'S') & (curSample['Type'] == 'U'))):
            curLiometopum=curSample['Hydrocarbon amount'].loc[curSample['Species']=='L']
            curSceptobius=curSample['Hydrocarbon amount'].loc[curSample['Species']=='S']
        
            p.circle(1,curLiometopum,color=colors[j],legend='Liometopum',line_width=4)
            p.circle(2,curSceptobius,color=colors[j],legend='Sceptobius',line_width=4)
            p.line([1,2],[curLiometopum,curSceptobius],color='black')
        
            source = ColumnDataSource(
                pd.DataFrame(
                    dict(
                        normalized_area=[curLiometopum.values],
                        position=[1],
                        labels=[str(i+1)]
                    )
                )
            )
        
            label=bokeh.models.LabelSet(x='position',y='normalized_area',text='labels',level='glyph',
                                       x_offset=-15, y_offset=-10, source=source, render_mode='canvas')
            p.add_layout(label)
                    
        if (any((curSample['Species'] == 'S') & (curSample['Type'] == 'L'))):
            curLiometopum=curSample['Hydrocarbon amount'].loc[curSample['Species']=='L']
            curSceptobius=curSample['Hydrocarbon amount'].loc[curSample['Species']=='S']
        
            p.circle(3,curLiometopum,color=colors[j],legend='Liometopum',line_width=4)
            p.circle(4,curSceptobius,color=colors[j],legend='Sceptobius',line_width=4)
            p.line([3,4],[curLiometopum,curSceptobius],color='black')
        
            source = ColumnDataSource(
                pd.DataFrame(
                    dict(
                        normalized_area=[curLiometopum.values],
                        position=[3],
                        labels=[str(i+1)]
                    )
                )
            )
        
            label=bokeh.models.LabelSet(x='position',y='normalized_area',text='labels',level='glyph',
                                       x_offset=-15, y_offset=-10, source=source, render_mode='canvas')
            p.add_layout(label)
        
        if (any((curSample['Species'] == 'P') & (curSample['Type'] == 'U'))):
            curLiometopum=curSample['Hydrocarbon amount'].loc[curSample['Species']=='L']
            curPlatyusa=curSample['Hydrocarbon amount'].loc[curSample['Species']=='P']
        
            p.circle(5,curLiometopum,color=colors[j],legend='Liometopum',line_width=4)
            p.circle(6,curPlatyusa,color=colors[j],legend='Platyusa',line_width=4)
            p.line([5,6],[curLiometopum,curPlatyusa],color='black')
        
            source = ColumnDataSource(
                pd.DataFrame(
                    dict(
                        normalized_area=[curLiometopum.values],
                        position=[5],
                        labels=[str(i+1)]
                    )
                )
            )
        
            label=bokeh.models.LabelSet(x='position',y='normalized_area',text='labels',level='glyph',
                                   x_offset=-20, y_offset=-10, source=source, render_mode='canvas')
            p.add_layout(label)
        
        if (any((curSample['Species'] == 'P') & (curSample['Type'] == 'L'))):
            curLiometopum=curSample['Hydrocarbon amount'].loc[curSample['Species']=='L']
            curPlatyusa=curSample['Hydrocarbon amount'].loc[curSample['Species']=='P']
        
            p.circle(7,curLiometopum,color=colors[j],legend='Liometopum',line_width=4)
            p.circle(8,curPlatyusa,color=colors[j],legend='Platyusa',line_width=4)
            p.line([7,8],[curLiometopum,curPlatyusa],color='black')
        
            source = ColumnDataSource(
                pd.DataFrame(
                    dict(
                        normalized_area=[curLiometopum.values],
                        position=[7],
                        labels=[str(i+1)]
                    )
                )
            )
        
            label=bokeh.models.LabelSet(x='position',y='normalized_area',text='labels',level='glyph',
                                       x_offset=-20, y_offset=-10, source=source, render_mode='canvas')
            p.add_layout(label)

p.legend.location = "top_left"


p.xaxis.visible = False
p.xgrid.visible = False
bokeh.io.show(p)

In [18]:
df_C24['Scaled Amount']=0
for _,date in enumerate(np.unique(df_C24['date'])):
    df_C24.loc[df_C24['date']==date,'Scaled Amount'] = preprocessing.MinMaxScaler().fit_transform(df_C24.loc[df_C24['date']==date,'Hydrocarbon amount'].values.reshape(-1, 1)).ravel()
    
df_C30['Scaled Amount']=0
for _,date in enumerate(np.unique(df_C30['date'])):
    df_C30.loc[df_C30['date']==date,'Scaled Amount'] = preprocessing.MinMaxScaler().fit_transform(df_C30.loc[df_C30['date']==date,'Hydrocarbon amount'].values.reshape(-1, 1)).ravel()

/Users/TomNaragon/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/TomNaragon/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/Users/TomNaragon/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = va

In [19]:
df_C24.head()

,Species,WellNumber,Peak ID,Ret Time,Start Tm,End Tm,m/z,Area,Area Percent,Height,Height Percent,A/H,Type,Outlier,date,Normalized Area,Hydrocarbon amount,Identifier,Scaled Amount
1,L,1,D50C24,9.498,9.475,9.525,TIC,231950,3.87,222378,3.55,1.04,L,N,190926,0.041494,1.037348,LSL,0.891385
4,S,1,D50C24,9.497,9.475,9.525,TIC,37683,0.62,31671,0.51,1.19,L,N,190926,0.006242,0.156048,SL,0.129477
7,L,2,D50C24,9.525,9.475,9.525,TIC,1607,0.03,756,0.01,2.13,U,N,190926,0.000251,0.006282,LSU,0.000000
10,S,2,D50C24,9.525,9.475,9.525,TIC,2344,0.04,1432,0.02,1.64,U,N,190926,0.000388,0.009708,SU,0.002962
13,L,3,D50C24,9.525,9.475,9.525,TIC,4472,0.08,2979,0.05,1.50,U,N,190926,0.000763,0.019070,LSU,0.011056


In [20]:
Identifiers = ['LSU','SU','LSL','SL','LPU','PU','LPL','PL','LDU','DU','LDL','DL',]
Labels={'LSU': '','SU': 'Sceptobius Control','LSL': '','SL': 'Sceptobius Treated',
        'LPU': '','PU': 'Platyusa Control','LPL': '','PL': 'Platyusa Treated',
        'LDU': '','DU': 'Dalotia Control','LDL': '','DL': 'Dalotia Treated'}

np.random.seed(666)  

p = bokeh.plotting.figure(plot_width=600,
                          plot_height=600,
                          title='C24',
                          x_range=Identifiers,
                          y_axis_label='ng D50C24')

colors=['#494949','#5F56FF','#494949','#5F56FF','#494949','#C42F2F','#494949','#C42F2F','#494949','#832161','#494949','#832161']


groups = df_C24.groupby('Identifier')
q1 = groups.quantile(q=0.25)
q2 = groups.quantile(q=0.5)
q3 = groups.quantile(q=0.75)
iqr = q3 - q1
upper = q3 + 1.5*iqr
lower = q1 - 1.5*iqr

# # find the outliers for each category
# def outliers(group):
#     cat = group.name
#     return group[(group['Scaled amount'] > upper.loc[cat]['Hydrocarbon amount']) | (group['Hydrocarbon amount'] < lower.loc[cat]['Hydrocarbon amount'])]['Hydrocarbon amount']
# out = groups.apply(outliers).dropna()

# # prepare outlier data for plotting, we need coordinates for every outlier.
# if not out.empty:
#     outx = []
#     outy = []
#     for keys in out.index:
#         outx.append(keys[0])
#         outy.append(out.loc[keys[0]].loc[keys[1]])

# # if no outliers, shrink lengths of stems to be no longer than the minimums or maximums
# qmin = groups.quantile(q=0.00)
# qmax = groups.quantile(q=1.00)
# upper['Hydrocarbon amount'] = [min([x,y]) for (x,y) in zip(list(qmax.loc[:,'Hydrocarbon amount']),upper['Hydrocarbon amount'])]
# lower['Hydrocarbon amount'] = [max([x,y]) for (x,y) in zip(list(qmin.loc[:,'Hydrocarbon amount']),lower['Hydrocarbon amount'])]

# # stems
# p.segment(q3.reset_index()['Identifier'], upper['Hydrocarbon amount'], q3.reset_index()['Identifier'], q3['Hydrocarbon amount'], line_color="black")
# p.segment(q3.reset_index()['Identifier'], lower['Hydrocarbon amount'], q3.reset_index()['Identifier'], q1['Hydrocarbon amount'], line_color="black")

# #boxes
# p.vbar(q3.reset_index()['Identifier'], 0.7, q2['Hydrocarbon amount'], q3['Hydrocarbon amount'], fill_color=None, line_color="black")
# p.vbar(q3.reset_index()['Identifier'], 0.7, q1['Hydrocarbon amount'], q2['Hydrocarbon amount'], fill_color=None, line_color="black")

# # whiskers (almost-0 height rects simpler than segments)
# p.vbar(q3.reset_index()['Identifier'], top=lower['Hydrocarbon amount'],bottom=lower['Hydrocarbon amount'], width=0.2, line_color="black")
# p.vbar(q3.reset_index()['Identifier'], top=upper['Hydrocarbon amount'],bottom=upper['Hydrocarbon amount'], width=0.2, line_color="black")

for j,date in enumerate(np.unique(df_C24['date'])):
    for _,i in enumerate(np.unique(df_C24['WellNumber'].loc[df_C24['date']==date])):
        
        data = {'identifier': df_C24['Identifier'].loc[(df_C24['date']==date)&(df_C24['WellNumber']==i)].values,
                'scaled amount': df_C24['Scaled Amount'].loc[(df_C24['date']==date)&(df_C24['WellNumber']==i)].values}

        source = ColumnDataSource(data=data)

        offsetVal=(np.random.rand(1)[0]-0.5)*0.5
#         p.line(bokeh.transform.dodge('identifier',  offsetVal,  range=p.x_range),
#                'scaled amount',
#                source=source,
#                color='black',
#                alpha=0.3)
        
        p.circle(bokeh.transform.dodge('identifier',  offsetVal,  range=p.x_range),
               'scaled amount',
               source=source,
               color=bokeh.transform.factor_cmap('identifier',colors,Identifiers),
               alpha=0.6,
               size=7)
        

p.xgrid.visible = False
p.ygrid.visible = False

# Add custom axis

p.xaxis.formatter = FuncTickFormatter(code="""
var labels = %s;
return labels[tick];
""" %Labels)


p.output_backend = "svg"
export_svgs(p, filename="DCHC_transfer_C24_minmax_scaled.svg")

bokeh.io.show(p)

In [21]:
Identifiers = ['LSU','SU','LSL','SL','LPU','PU','LPL','PL','LDU','DU','LDL','DL',]
Labels={'LSU': '','SU': 'Sceptobius Control','LSL': '','SL': 'Sceptobius Treated',
        'LPU': '','PU': 'Platyusa Control','LPL': '','PL': 'Platyusa Treated',
        'LDU': '','DU': 'Dalotia Control','LDL': '','DL': 'Dalotia Treated'}

np.random.seed(666)  

p = bokeh.plotting.figure(plot_width=600,
                          plot_height=600,
                          title='C30',
                          x_range=Identifiers,
                          y_axis_label='ng D50C24')

colors=['#494949','#5F56FF','#494949','#5F56FF','#494949','#C42F2F','#494949','#C42F2F','#494949','#832161','#494949','#832161']


groups = df_C30.groupby('Identifier')
q1 = groups.quantile(q=0.25)
q2 = groups.quantile(q=0.5)
q3 = groups.quantile(q=0.75)
iqr = q3 - q1
upper = q3 + 1.5*iqr
lower = q1 - 1.5*iqr


for j,date in enumerate(np.unique(df_C30['date'])):
    for _,i in enumerate(np.unique(df_C30['WellNumber'].loc[df_C30['date']==date])):
        
        data = {'identifier': df_C30['Identifier'].loc[(df_C30['date']==date)&(df_C30['WellNumber']==i)].values,
                'scaled amount': df_C30['Scaled Amount'].loc[(df_C30['date']==date)&(df_C30['WellNumber']==i)].values}

        source = ColumnDataSource(data=data)

        offsetVal=(np.random.rand(1)[0]-0.5)*0.5
#         p.line(bokeh.transform.dodge('identifier',  offsetVal,  range=p.x_range),
#                'scaled amount',
#                source=source,
#                color='black',
#                alpha=0.3)
        
        p.circle(bokeh.transform.dodge('identifier',  offsetVal,  range=p.x_range),
               'scaled amount',
               source=source,
               color=bokeh.transform.factor_cmap('identifier',colors,Identifiers),
               alpha=0.6,
               size=7)
        

p.xgrid.visible = False

# Add custom axis

p.xaxis.formatter = FuncTickFormatter(code="""
var labels = %s;
return labels[tick];
""" %Labels)

bokeh.io.show(p)